<a href="https://colab.research.google.com/github/cbonnin88/The_Coffee_Corner/blob/main/the_coffee_corner_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Loading our datasets
orders = pd.read_csv('coffee_shop - clean_orders.csv')
products = pd.read_csv('coffee_shop - products.csv')
customers = pd.read_csv('coffee_shop - customers.csv')

In [ ]:
# Function to clean currency strings
def clean_currency(x):
  if isinstance(x,str):
    return float(x.replace('€','').replace(',','.').strip())
  return x

In [ ]:
# Applying our cleaning function
orders['total_sale'] = orders['total_sale'].apply(clean_currency)
orders['order_date']= pd.to_datetime(orders['order_date'])
products['profit'] = products['profit'].apply(clean_currency)

In [ ]:
# Merging for full context
df_coffee = orders.merge(products, on='product_reference_number',suffixes=('_ord','_prod'))
df_coffee_final = df_coffee.merge(customers,on='customer_reference_number')

In [ ]:
df_coffee_final = df_coffee_final.rename(columns={'country_x':'country'})

In [ ]:
df_coffee_final.columns

Index(['order_reference_number', 'order_id', 'order_date',
       'customer_reference_number', 'product_reference_number', 'quantity',
       'unit_price_ord', 'total_sale', 'country', 'year', 'coffee_type_ord',
       'product_id', 'coffee_type_prod', 'roast_type', 'size',
       'unit_price_prod', 'price_per_kg', 'profit', 'external_customer_id',
       'first_name', 'last_name', 'email', 'phone_number', 'address', 'city',
       'country_y', 'postcode', 'loyalty_card'],
      dtype='object')

In [ ]:
print(f'Total Rows: {df_coffee_final.shape[0]}')

Total Rows: 17


In [ ]:
display(df_coffee_final[['order_date','total_sale','country','coffee_type_ord']].head())

,order_date,total_sale,country,coffee_type_ord
0,2019-12-29,7.77,France,Ara
1,2020-06-07,8.91,United Kingdom,Exc
2,2019-03-04,22.89,United Kingdom,Rob
3,2019-06-17,9.51,France,Lib
4,2021-04-05,4.46,France,Exc


In [ ]:
df_coffee_final.head()

,order_reference_number,order_id,order_date,customer_reference_number,product_reference_number,quantity,unit_price_ord,total_sale,country,year,...,external_customer_id,first_name,last_name,email,phone_number,address,city,country_y,postcode,loyalty_card
0,2286,KTX-17944-494,2019-12-29,1543,3964,1,"7,77 €",7.77,France,2019,...,41611-34336-WT,Peyter,Lauritzen,plauritzengo@photobucket.com,+33 4 88 80 02 00,"21, rue Thibaut Mallet",Nantes,France,44000,Yes
1,2274,SHP-17012-870,2020-06-07,1904,3889,1,"8,91 €",8.91,United Kingdom,2020,...,22721-63196-UJ,Gladi,Ducker,gduckerdx@patch.com,+01 1 44 96 02 67,5069 Boyd Parkway,London,United Kingdom,E1 7PZ,No
2,2033,CGO-79583-871,2019-03-04,1649,3710,1,"22,89 €",22.89,United Kingdom,2019,...,66240-46962-IO,Berty,Beelby,bbeelbyej@rediff.com,+441184960644,844 Sachs Avenue,London,United Kingdom,E1 7TL,No
3,2341,LCU-93317-340,2019-06-17,1807,3924,1,"9,51 €",9.51,France,2019,...,20118-28138-QD,Jaquenette,Skentelbery,jskentelberyoa@paypal.com,+33 1 66 21 21 49,"8, rue Bertin",Paris,France,75018,Yes
4,2346,DCE-22886-861,2021-04-05,1702,3658,1,"4,46 €",4.46,France,2021,...,49671-11547-WG,Jacquelyn,Maha,jmahakc@cyberchimps.com,+33 5 44 48 64 92,"6, chemin de Boutin",Paris,France,75012,Yes


In [ ]:
print(f'total Sales in Dataset: €{df_coffee_final['total_sale'].sum():,.2f}')

total Sales in Dataset: €515.06


# **Exploratory Product Data Analysis (EPDA)**

**Question 1:** How does our sales trend over time ?

In [ ]:
# Aggregate data by week
weekly_sales = df_coffee_final.resample('W', on='order_date')['total_sale'].sum().reset_index()

display(weekly_sales)

,order_date,total_sale
0,2019-03-10,22.89
1,2019-03-17,0.00
2,2019-03-24,0.00
3,2019-03-31,0.00
4,2019-04-07,0.00
...,...,...
171,2022-06-19,0.00
172,2022-06-26,0.00
173,2022-07-03,44.55
174,2022-07-10,0.00


In [ ]:
# Creating our line chart
weekly_sale_chart = px.line(
    weekly_sales,
    x='order_date',
    y='total_sale',
    title='Weekly Sales Trends (2019-2022)',
    labels={'total_sale':'Revenue (€)','order_date':'Date'})

weekly_sale_chart.show()

**Question 2:** Which coffee type is the most popular

In [ ]:
# Grouping by Coffee Type
coffee_popularity = df_coffee_final.groupby('coffee_type_ord')['quantity'].sum().reset_index()

display(coffee_popularity)

,coffee_type_ord,quantity
0,Ara,7
1,Exc,12
2,Lib,6
3,Rob,12


In [ ]:
coffee_bar_chart = px.bar(
    coffee_popularity,
    x='coffee_type_ord',
    y='quantity',
    color='coffee_type_ord',
    title='Total Quantity Sold by Coffee Type',
    labels={'quantity':'Unit Sold','coffee_type_ord':'Coffee Type'}
)

coffee_bar_chart.show()

**Question 3:** Who are our top 5 countries by Profit

In [ ]:
# Calculate Total Profit per order (Quantity * Unit Profit)
df_coffee_final['total_profit'] = df_coffee_final['quantity'] * df_coffee_final['profit']

display(df_coffee_final['total_profit'].head())

,total_profit
0,0.70
1,0.98
2,1.37
3,1.24
4,0.49


In [ ]:
# Group by Country
country_profit = df_coffee_final.groupby('country')['total_profit'].sum().reset_index()
country_profit = country_profit.sort_values(by='total_profit', ascending=False).head(5)

display(country_profit)

,country,total_profit
0,France,40.92
1,United Kingdom,4.79


In [ ]:
country_profit_chart = px.bar(
    country_profit,
    x='country',
    y='total_profit',
    title='Top Countries by Total Profit',
    text_auto='.2s',
    color='total_profit'
)

country_profit_chart.show()

**Question 4:** Is there a correlation between the unit price and the quantity ?

In [ ]:
correlation_scatter = px.scatter(
    df_coffee_final,
    x='unit_price_ord',
    y='quantity',
    color='coffee_type_ord',
    size='total_sale',
    title='Correlation: Unit Price vs. Quantity',
    labels={'unit_price_ord':'Unit Price (€)','quantity':'Quantity'},
    hover_data=['product_id']
)

correlation_scatter.show()